In [1]:
import CRUD
import pandas as pd

In [4]:
connection = CRUD.connectToDatabase("./data/ikon.db")
sql = '''
SELECT e.id, e.title, e.date, g.name
FROM exhibitions e
JOIN galleries g ON g.id = e.gallery_id
ORDER BY date DESC, title ASC;
'''

data = connection.execute(sql).fetchall()
df = pd.DataFrame(data, columns=['id', 'title', 'date', 'gallery'])

In [6]:
# Create events column
events = [
    "vezetés",
    "tárlatvezetés",
    "finisszázs",
    "finissage",
    "előadás",
    "beszélgetés",
    "interjú",
    "könyvbemutató",
    "workshop",
    "bemutató",
    "pályázat",
    "felhívás"
]

for idx, row in df.iterrows():
    for event in events:
        if event in row['title'].lower():
            df.loc[idx, 'event'] = event
            

,id,title,date,gallery,event
0,14867,"RAJK LÁTVÁNY, tárlatvezetés",2020-01-05,Centrális Galéria,tárlatvezetés
1,14866,Hangover Reading Club #4 - Let's go with the f...,2020-01-04,Trafó Galéria,NaN
2,14868,"RAJK LÁTVÁNY, tárlatvezetés",2020-01-04,Centrális Galéria,tárlatvezetés
3,14870,BONNE ANNÉE BONNE SANTÉ! NAGY FORMÁTUMOK KIS F...,2020-01-03,Három Hét Galéria,NaN
4,14872,Rendetlenség a rendben | tárlatvezetés a gyűjt...,2020-01-01,Fővárosi Képtár - Kiscelli Múzeum,tárlatvezetés
...,...,...,...,...,...
14867,29,Ékszertől a plasztikáig - kortárs magyar ötvös...,2000-01-01,Iparművészeti Múzeum,NaN
14868,54,Ü V E G E K,2000-01-01,Godot Galéria,NaN
14869,13,Expedíció a jövőbe,1999-12-31,Centrális Galéria,NaN
14870,2,Actionman,1999-12-07,Fiatal Képzőművészek Stúdiója - Stúdió Galéria,NaN


In [15]:
import ipywidgets as widgets
progressbar = widgets.IntProgress(min=0, max=len(df))
display(progressbar)
for i in range(len(df)):
    idx = df.index[i]
    row = df.iloc[i]
    progressbar.value+=1
    artists = CRUD.select_artists_of_exhibition(connection, (idx, ) )
    names = "; ".join([a[1] for a in artists])
    df.loc[idx, 'artists'] = names
    

IntProgress(value=0, max=14872)

In [20]:
df[['id', 'title', 'date', 'gallery', 'artists']]

,id,title,date,gallery,artists
0,14867,"RAJK LÁTVÁNY, tárlatvezetés",2020-01-05,Centrális Galéria,
1,14866,Hangover Reading Club #4 - Let's go with the f...,2020-01-04,Trafó Galéria,Vajda Lajos Stúdió; Ef Zámbó István
2,14868,"RAJK LÁTVÁNY, tárlatvezetés",2020-01-04,Centrális Galéria,Kupcsik Adrián; Bencsik Barnabás
3,14870,BONNE ANNÉE BONNE SANTÉ! NAGY FORMÁTUMOK KIS F...,2020-01-03,Három Hét Galéria,Korok; -; -
4,14872,Rendetlenség a rendben | tárlatvezetés a gyűjt...,2020-01-01,Fővárosi Képtár - Kiscelli Múzeum,L; -; -
...,...,...,...,...,...
14867,29,Ékszertől a plasztikáig - kortárs magyar ötvös...,2000-01-01,Iparművészeti Múzeum,Rajk László
14868,54,Ü V E G E K,2000-01-01,Godot Galéria,Rajk László
14869,13,Expedíció a jövőbe,1999-12-31,Centrális Galéria,Művészeti foglalkozás
14870,2,Actionman,1999-12-07,Fiatal Képzőművészek Stúdiója - Stúdió Galéria,Szöllősi-Nagy - Nemes Gyűjtemény; Szöllősi-Nag...
